In [ ]:
from sklearn.svm import SVC

#Zaimplementowane metody
from StratifiedBagging import StratifiedBagging
from implementedBoosting import implementedBoosting
from RandomSubspace import RandomSubspace
from RandomPatches import RandomPatches

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.base import clone


In [ ]:
 clfs = {
     'SVC' : SVC(probability=True, random_state=42),
     'sBag': StratifiedBagging(),    
     'BST': implementedBoosting(base_estimator = SVC(probability=True, random_state=42), n_estimators = 10),
     'RSP' : RandomSubspace(base_estimator = SVC(probability=True, random_state=42), n_estimators = 10, random_state=42),
     'RPT' : RandomPatches(),
 }

In [ ]:
datasets = ['australian', 'wisconsin', 'breastcan', 'cryotherapy', 'diabetes',
           'heart', 'ionosphere', 'liver', 'monkthree', 'sonar',
           'bupa', 'monk-2', 'heart-failure', 'wine-2', 'wdbc',
           'banknote', 'twonorm', 'Heart-Disease', 'pima', 'cardiovascular',]

In [ ]:
n_datasets = len(datasets)
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=42)

scores = np.zeros((len(clfs), n_datasets, n_splits * n_repeats))

In [ ]:
for data_id, dataset in enumerate(datasets):
    dataset = np.genfromtxt("datasets/%s.csv" % (dataset), delimiter=",")
    X = dataset[:, :-1]
    y = dataset[:, -1].astype(int)

    for fold_id, (train, test) in enumerate(rskf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])
            clf.fit(X[train], y[train])
            y_pred = clf.predict(X[test])
            scores[clf_id, data_id, fold_id] = accuracy_score(y[test], y_pred)

np.save('results', scores)

In [ ]:
mean_scores = np.mean(scores, axis=2).T
print("\nMean scores:\n", mean_scores)
np.save('results', scores)